In [1]:
%load_ext autoreload

%autoreload 1

In [2]:
import nest_asyncio
nest_asyncio.apply()

In [3]:
import os
from pprint import pprint

In [4]:
from akd.config import CONFIG

2025-04-01 20:48:27.228 | INFO     | akd.config:get_config:78 - Loading configuration for environment: LOCAL


In [5]:
# CONFIG

# Import

In [6]:
from akd.tools.search import (
    SearxNGSearchTool,
    SearxNGSearchToolConfig,
    SearxNGSearchToolInputSchema
)

In [7]:
from akd.tools.scrapers.web_scrapers import (
    SimpleWebScraper,
    WebpageScraperToolConfig,
    WebpageScraperToolInputSchema,
    Crawl4AIWebScraper
)
from akd.tools.scrapers.pdf_scrapers import (
    SimplePDFScraper,
)
from akd.tools.scrapers.composite import CompositeWebScraper

In [8]:
from akd.tools.scrapers.resolvers import (
    ArxivResolver,
    ADSResolver,
    IdentityResolver,
    ResolverInputSchema
)

from akd.tools.scrapers.composite import ResearchArticleResolver

In [9]:
from akd.agents.factory import (
    create_intent_agent,
    create_extraction_agent,
    create_intent_agent,
    create_query_agent
)
from akd.agents.extraction import (
    IntentBasedExtractionSchemaMapper, 
    EstimationExtractionAgent,
    ExtractionInputSchema,
)

In [10]:
from akd.agents.litsearch import LitAgent, LitAgentInputSchema

In [11]:
from akd.agents.intents import Intent, IntentInputSchema, IntentAgent
from akd.agents._base import BaseAgentConfig

In [12]:
import instructor
import openai

# Build Tools and Agents

In [13]:
SEARCH_TOOL = SearxNGSearchTool(
    config=SearxNGSearchToolConfig(
        base_url="http://localhost:8080",
        max_results=5,
        engines=["google", "arxiv", "google_scholar"],
        debug=True
    )
)

In [14]:
await SEARCH_TOOL.arun(
    SearxNGSearchToolInputSchema(
        queries=[
            "landslides nepal"
        ]
    )
)

2025-04-01 20:48:29.084 | DEBUG    | akd.tools.search:arun:259 - [{'template': 'paper.html', 'type': 'pdf', 'url': 'https://nhess.copernicus.org/articles/18/3203/2018/', 'title': 'Roads and landslides in Nepal: how development affects environmental risk - (Published 2018-01-01T00:00:00)', 'authors': ['BG McAdoo', 'M Quak', 'KR Gnyawali…'], 'publisher': None, 'journal': '… Hazards and Earth …', 'publishedDate': '2018-01-01T00:00:00', 'content': '… and landslides in the Sindhupalchok District of central Nepal (Fig. 3). The 2015 Gorkha earthquake heavily impacted Sindhupalchok, where over 95 % of the houses were severely …', 'comments': '', 'html_url': None, 'pdf_url': 'https://nhess.copernicus.org/articles/18/3203/2018/nhess-18-3203-2018.pdf', 'engine': 'google scholar', 'parsed_url': ['https', 'nhess.copernicus.org', '/articles/18/3203/2018/', '', '', ''], 'engines': ['google scholar'], 'positions': [1], 'score': 1.0, 'category': 'science', 'query': 'landslides nepal'}, {'template': 'pa

SearxNGSearchToolOutputSchema(results=[SearchResultItem(url=HttpUrl('https://nhess.copernicus.org/articles/18/3203/2018/'), pdf_url=HttpUrl('https://nhess.copernicus.org/articles/18/3203/2018/nhess-18-3203-2018.pdf'), title='Roads and landslides in Nepal: how development affects environmental risk - (Published 2018-01-01T00:00:00)', content='… and landslides in the Sindhupalchok District of central Nepal (Fig. 3). The 2015 Gorkha earthquake heavily impacted Sindhupalchok, where over 95 % of the houses were severely …', query='landslides nepal', category='science', doi=None, published_date='2018-01-01T00:00:00', engine='google scholar', tags=None, extra={'template': 'paper.html', 'type': 'pdf', 'authors': ['BG McAdoo', 'M Quak', 'KR Gnyawali…'], 'publisher': None, 'journal': '… Hazards and Earth …', 'comments': '', 'html_url': None, 'parsed_url': ['https', 'nhess.copernicus.org', '/articles/18/3203/2018/', '', '', ''], 'engines': ['google scholar'], 'positions': [1], 'score': 1.0}), Sea

In [15]:
scraper_cfg = WebpageScraperToolConfig()
SCRAPER = CompositeWebScraper(
    SimpleWebScraper(scraper_cfg),
    Crawl4AIWebScraper(scraper_cfg),
    SimplePDFScraper(scraper_cfg),
    debug=True
)

In [16]:
_result = await SCRAPER.arun(
    WebpageScraperToolInputSchema(
        # url="https://www.sciencedirect.com/science/article/pii/S0048969722074824"
        url="https://www.wordstemplates.org/wp-content/uploads/2021/06/Statement-of-Purpose-Format.pdf"
    )
)

2025-04-01 20:48:29.106 | DEBUG    | akd.tools.scrapers.composite:arun:39 - Running scraper=SimpleWebScraper for {"url":"https://www.wordstemplates.org/wp-content/uploads/2021/06/Statement-of-Purpose-Format.pdf","include_links":true}
2025-04-01 20:48:29.107 | ERROR    | akd.tools.scrapers.composite:arun:48 - Error running SimpleWebScraper
Can't parse url with PDF :: https://www.wordstemplates.org/wp-content/uploads/2021/06/Statement-of-Purpose-Format.pdf
2025-04-01 20:48:29.108 | DEBUG    | akd.tools.scrapers.composite:arun:39 - Running scraper=Crawl4AIWebScraper for {"url":"https://www.wordstemplates.org/wp-content/uploads/2021/06/Statement-of-Purpose-Format.pdf","include_links":true}
2025-04-01 20:48:29.108 | ERROR    | akd.tools.scrapers.composite:arun:48 - Error running Crawl4AIWebScraper
Can't parse url with PDF :: https://www.wordstemplates.org/wp-content/uploads/2021/06/Statement-of-Purpose-Format.pdf
2025-04-01 20:48:29.109 | DEBUG    | akd.tools.scrapers.composite:arun:39 - Ru

In [17]:
_result.content

'Statement of Purpose\nPhilip Guo (pgbovine@mit.edu)\nI want to pursue a Ph.D. in Computer Science, and my career aspiration is to become\na professor. My research interests include program analysis, software engineering, operating\nsystems, and human-computer interaction (HCI).\nMy motivation for pursuing research related to program analysis stems from learning\nabout the challenges of software development during 3 summers spent as a software engi-\nneering intern (1 at Codehost, a start-up, and 2 at Teradyne, a large corporation). Although\nI found the actual development work to be educational, what really sparked my interest in\nresearch was experiencing ﬁrst-hand the challenges that software engineers face daily.\nI aspire to create tools that can increase programmer productivity and improve software\nquality and security. As a ﬁrst step towards this goal, for the past 2 years I have been working\nwith Prof. Michael Ernst in the MIT Program Analysis Group. The research problem that

In [18]:
ARTICLE_RESOLVER = ResearchArticleResolver(
    ArxivResolver(),
    ADSResolver(),
    IdentityResolver()
)

In [19]:
await ARTICLE_RESOLVER.arun(
    ResolverInputSchema(url="https://arxiv.org/abs/2411.08181")
)

2025-04-01 20:48:29.451 | DEBUG    | akd.tools.scrapers.composite:resolve:69 - Using resolver=ArxivResolver for url=https://arxiv.org/abs/2411.08181


ResolverOutputSchema(url=HttpUrl('https://arxiv.org/pdf/2411.08181.pdf'), resolver='ArxivResolver')

In [20]:
# INTENT_AGENT = create_intent_agent()

INTENT_AGENT = IntentAgent(
    config=BaseAgentConfig(
        client=instructor.from_openai(openai.AsyncOpenAI())
    )
)

In [21]:
isinstance(INTENT_AGENT.client, instructor.client.AsyncInstructor)

True

In [22]:
await INTENT_AGENT.arun(
    IntentInputSchema(
        query="landslide nepal"
    )
)

IntentOutputSchema(intent=<Intent.GENERAL: 'General'>)

In [23]:
await INTENT_AGENT.arun(
    IntentInputSchema(
        query="landslide nepal"
    )
)

IntentOutputSchema(intent=<Intent.GENERAL: 'General'>)

In [24]:
QUERY_AGENT = create_query_agent()

In [25]:
SCHEMA_MAPPER = IntentBasedExtractionSchemaMapper()

In [26]:
await SCHEMA_MAPPER.arun(Intent.ESTIMATION)

typing.List[akd.structures.SingleEstimation]

In [27]:
EXTRACTION_AGENT = EstimationExtractionAgent()

In [30]:
# _result = EXTRACTION_AGENT.run(
#     ExtractionInputSchema(
#         query="Hello i am nishan pantha.",
#         content="I am nishan. But call me Nish. You can find me at example.com"
#     )
# )

In [31]:
# _result.estimations

In [32]:
lit_agent = LitAgent(
    intent_agent=INTENT_AGENT,
    schema_mapper=IntentBasedExtractionSchemaMapper(),
    query_agent=QUERY_AGENT,
    extraction_agent=EXTRACTION_AGENT,
    search_tool=SEARCH_TOOL,
    web_scraper=SCRAPER,
    article_resolver=ARTICLE_RESOLVER,
)
lit_agent.clear_history()

2025-04-01 20:49:28.953 | WARNING  | akd.agents.litsearch:clear_history:141 - Clearing history for all the agents


In [33]:
lit_agent.memory

In [34]:
# query = "mangrove forest statuses in cameroon"
# query = "flash flood in south east asia land cover"
query = "methods and estimation to map landslides in Nepal"

In [35]:
result = await lit_agent.arun(
    LitAgentInputSchema(query=query)
)

2025-04-01 20:49:33.937 | INFO     | akd.agents.litsearch:arun:83 - Analyzing input query to generate relevant search queries...
2025-04-01 20:49:35.066 | DEBUG    | akd.agents.litsearch:arun:88 - Generated search queries:
2025-04-01 20:49:35.068 | DEBUG    | akd.agents.litsearch:arun:90 - Query 1: methods and estimation to map landslides in Nepal
2025-04-01 20:49:35.068 | DEBUG    | akd.agents.litsearch:arun:90 - Query 2: methods for mapping landslides in Nepal
2025-04-01 20:49:35.069 | DEBUG    | akd.agents.litsearch:arun:90 - Query 3: landslide estimation techniques Nepal
2025-04-01 20:49:35.070 | DEBUG    | akd.agents.litsearch:arun:90 - Query 4: remote sensing landslide mapping Nepal
2025-04-01 20:49:35.071 | INFO     | akd.agents.litsearch:arun:93 - Searching across the web using SearxNG...
2025-04-01 20:49:40.104 | DEBUG    | akd.tools.search:arun:259 - [{'template': 'paper.html', 'type': 'html', 'url': 'https://idp.springer.com/authorize/casa?redirect_uri=https://link.springer.

[INIT].... → Crawl4AI 0.5.0.post4
[FETCH]... ↓ https://hdl.handle.net/20.500.14017/8df80ddb-1fa4-... | Status: True | Time: 4.81s
[SCRAPE].. ◆ https://hdl.handle.net/20.500.14017/8df80ddb-1fa4-... | Time: 0.013s
[COMPLETE] ● https://hdl.handle.net/20.500.14017/8df80ddb-1fa4-... | Status: True | Total: 4.83s


/Users/npantha/dev/impact/projects/accelerated-discovery/.venv/lib/python3.12/site-packages/bs4/builder/__init__.py:409: RuntimeWarning: coroutine 'AsyncInstructor.create' was never awaited
  universal: Set[str] = self.cdata_list_attributes.get("*", set())
2025-04-01 20:49:55.708 | DEBUG    | akd.agents.litsearch:arun:122 - Result 4: Landslide susceptibility mapping using the weight of evidence method in the Tinau watershed, Nepal | https://hdl.handle.net/20.500.14017/8df80ddb-1fa4-4abd-929e-79249a9b579a |   * [Skip to main navigation](https://hdl.handle.net/20.500.14017/8df80ddb-1fa4-4abd-929e-79249a9b57.. | words=1265
2025-04-01 20:49:55.709 | DEBUG    | akd.agents.litsearch:arun:105 - Result 5 : Scraping the url https://idp.springer.com/authorize/casa?redirect_uri=https://link.springer.com/article/10.1007/s12665-021-09650-2&casa_token=KyT2vJyOo1MAAAAA:edXp_J36TuFD_zii4p_L2xrTIgN25ty01l0AMHaXCvN4QSRv46JzXS9Al6m0W-TT3MiutGFvxIkYFhE
2025-04-01 20:49:55.709 | DEBUG    | akd.tools.scrape

In [39]:
len(result)

5

In [40]:
result

[ExtractionDTO(source='https://idp.springer.com/authorize/casa?redirect_uri=https://link.springer.com/article/10.1007/s12665-021-09650-2&casa_token=H7T6_At-PnYAAAAA:WVHQdzme1Mai1_VW1CJ5grwxbwuy9McmgOuCEq0B8OUyFFUPKiTTMNSPld6vvQa905gOH1Wbhp-BBFk', result=EstimationExtractionOutputSchema(estimations=[SingleEstimation(answer='The artificial neural network approach yielded the best prediction capability with an AUC value of 86.9% for landslide susceptibility mapping in the high mountainous region of Nepal.', related_knowledge=['Landslide susceptibility mapping (LSM) helps identify landslide-prone areas.', 'The study compared four methods: frequency ratio, logistic regression, artificial neural network, and support vector machine.'], research_data=ResearchData(data_format='PDF', origin='Landslide susceptibility mapping with GIS in high mountain area of Nepal: a comparison of four methods', data_url=HttpUrl('https://doi.org/10.1007/s12665-021-09650-2')), methodology='The study employed four 

In [41]:
result[0].result.estimations

[SingleEstimation(answer='The artificial neural network approach yielded the best prediction capability with an AUC value of 86.9% for landslide susceptibility mapping in the high mountainous region of Nepal.', related_knowledge=['Landslide susceptibility mapping (LSM) helps identify landslide-prone areas.', 'The study compared four methods: frequency ratio, logistic regression, artificial neural network, and support vector machine.'], research_data=ResearchData(data_format='PDF', origin='Landslide susceptibility mapping with GIS in high mountain area of Nepal: a comparison of four methods', data_url=HttpUrl('https://doi.org/10.1007/s12665-021-09650-2')), methodology='The study employed four approaches for LSM: frequency ratio, logistic regression, artificial neural network, and support vector machine, using various landslide causal factors.', assumptions=['The conditions that led to past landslides will similarly affect future landslides.'], confidence_level=86.9, validation_method='T

# Display result

In [42]:
from typing import List, TypeVar, Any
from pydantic import BaseModel
import json
from pprint import pformat

T = TypeVar('T', bound=BaseModel)

In [43]:
def display_result(models_list: List[T], indent: int = 2) -> None:
    """
    Alternative version using model_dump_json method directly.
    
    Args:
        models_list: A list of Pydantic model instances
        indent: Number of spaces for indentation (default: 2)
    """
    if not models_list:
        print("[]")
        return
    
    print(f"[{len(models_list)} items]")
    
    for i, model in enumerate(models_list):
        # Use Pydantic's built-in JSON serialization
        formatted_model = model.model_dump_json(indent=indent)
        
        print(f"\nItem {i + 1}:")
        print(formatted_model)

In [44]:
for res in result:
    print(f"{res.source}")
    display_result(res.result.estimations)
    print("-"*42)

https://idp.springer.com/authorize/casa?redirect_uri=https://link.springer.com/article/10.1007/s12665-021-09650-2&casa_token=H7T6_At-PnYAAAAA:WVHQdzme1Mai1_VW1CJ5grwxbwuy9McmgOuCEq0B8OUyFFUPKiTTMNSPld6vvQa905gOH1Wbhp-BBFk
[1 items]

Item 1:
{
  "answer": "The artificial neural network approach yielded the best prediction capability with an AUC value of 86.9% for landslide susceptibility mapping in the high mountainous region of Nepal.",
  "related_knowledge": [
    "Landslide susceptibility mapping (LSM) helps identify landslide-prone areas.",
    "The study compared four methods: frequency ratio, logistic regression, artificial neural network, and support vector machine."
  ],
  "research_data": {
    "data_format": "PDF",
    "origin": "Landslide susceptibility mapping with GIS in high mountain area of Nepal: a comparison of four methods",
    "data_url": "https://doi.org/10.1007/s12665-021-09650-2"
  },
  "methodology": "The study employed four approaches for LSM: frequency ratio, l

# Relevancy Check

In [45]:
from akd.tools.relevancy import (
    RelevancyChecker,
    RelevancyCheckerConfig,
    RelevancyCheckerInputSchema,
    _RelevancyCheckerSwappedInputSchema
)
from akd.agents.relevancy import RelevancyAgent

In [46]:
relevancy_checker = RelevancyChecker(
    RelevancyCheckerConfig(
        debug=True,
    )
)

In [47]:
relevancy_result = await relevancy_checker.arun(
    RelevancyCheckerInputSchema(
        query=query,
        content=str(result[0])
    )
)

2025-04-01 20:51:22.825 | INFO     | akd.tools.relevancy:arun:90 - Running relevancy check for query: methods and estimation to map landslides in Nepal
2025-04-01 20:51:24.634 | DEBUG    | akd.tools.relevancy:_run:110 - Relevancy check 1: {"label":"Relevant","reasoning_steps":["The query specifically asks about methods and estimation for mapping landslides in Nepal.","The content provides detailed information about a study that compares different methods for landslide susceptibility mapping in Nepal.","It includes specific methodologies, estimations, and validation methods relevant to the query."]}
2025-04-01 20:51:25.963 | DEBUG    | akd.tools.relevancy:_run:110 - Relevancy check 2: {"label":"Relevant","reasoning_steps":["The query specifically asks about methods and estimation for mapping landslides in Nepal.","The content provides detailed information about a study that compares different methods for landslide susceptibility mapping in Nepal.","It includes specific methodologies, es

In [48]:
pprint(relevancy_result.model_dump())

{'reasoning_steps': ['The query specifically asks about methods and estimation '
                     'for mapping landslides in Nepal.',
                     'The content provides detailed information about a study '
                     'that compares different methods for landslide '
                     'susceptibility mapping in Nepal.',
                     'It includes specific methodologies, estimations, and '
                     'validation methods relevant to the query.',
                     'The query specifically asks about methods and estimation '
                     'for mapping landslides in Nepal.',
                     'The content provides detailed information about a study '
                     'that compares different methods for landslide '
                     'susceptibility mapping in Nepal.',
                     'It includes specific methodologies, estimations, and '
                     'validation methods relevant to the query.',
                     'Th